<a href="https://colab.research.google.com/github/jmanning21/31005_MachineLearning_Python/blob/master/Machine_Learning_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Machine Learning Project
####Affnan Amir (13528841) and Julia Manning (12875795)

###Introduction
The algorithm chosen for this project is the ID3 Decision Tree building algorithm. In using this alrogithm, the data  

* Input = titanic dataset from given url:
https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
* Output = predicted resutls for those individuals which survived (i.e. binary options; Did they survive? Yes or No).

###Exploration

Challenges:
 - over-fitting the data

###Methodology

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [0]:
# Function importing dataset and cleaning
def importdata(): 
  url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
  # Read the csv and convert to a dataframe
  d = pd.read_csv(url)
  df = pd.DataFrame(data=d)

  median_values = df.groupby(['Pclass','Sex'])['Age'].transform('median')
  df['Age'].fillna(median_values, inplace=True)
  
  # Adding and removing specific columns which may help with the accuracy of the accuracy.
  # Add a Child.Adult column based on the age of the individual
  df['Child.Adult'] = 'Adult'

  # Child = Age < 13 and Adult = Age >= 13 
  df['Child.Adult'][df['Age'] < 13.0] = 'Child'

  # Remove 'Name' column as it does not provide any relevant information to the dataset, as all the values are unique
  del df['Name']
  # Remove 'Cabin' as there are too many NA values and the remaining values are all unique, make it impossible to determine a suitable replacement for the NA values
  del df['Cabin']

  # Re-ordering the columns to put the 'Survived' column last
  df = df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked','Child.Adult','Survived']]
  
  df['Sex'],_ = pd.factorize(df['Sex'])
  df['Embarked'],_ = pd.factorize(df['Embarked'])
  df['Child.Adult'],_ = pd.factorize(df['Child.Adult'])
  df['Ticket'],_ = pd.factorize(df['Ticket'])
  
  return df

In [0]:
# function to split the data
def splitdataset(df,split):
  # Seperating the target variable 
    X = df.iloc[:,:-1]
    Y = df.iloc[:,-1] 
    test_split = 1 - split
    
    # Spliting the dataset into train and test 
    X_train, X_test, y_train, y_test = train_test_split(  
    X, Y, test_size = test_split, random_state = 100) 
    
    return X, Y, X_train, X_test, y_train, y_test

In [0]:
# Function to perform training with giniIndex. 
def train_using_gini(X_train, X_test, y_train): 
  
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(
        criterion = "gini", random_state = 123,
        max_depth=3, min_samples_leaf=5) 
  
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 

In [0]:
# Function to perform training with entropy. 
def train_using_entropy(X_train, X_test, y_train): 
  
    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 123, 
            max_depth = 3, min_samples_leaf = 5) 
  
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy

In [0]:
# Function to make predictions 
def prediction(X_test, clf_object): 
  
    # Predicton on test with giniIndex 
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:") 
    print(y_pred) 
    return y_pred

In [0]:
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: ", 
        confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : {:.2f}%".format(accuracy_score(y_test,y_pred)*100)) 
            
    print("Report : ", 
    classification_report(y_test, y_pred))

In [0]:
# Driver code 
def main(var): 
      
    # Building Phase 
    data = importdata()
    X, Y, X_train, X_test, y_train, y_test = splitdataset(data,var) 
    clf_gini = train_using_gini(X_train, X_test, y_train) 
    clf_entropy = train_using_entropy(X_train, X_test, y_train) 
      
    # Operational Phase 
    print("Results Using Gini Index:") 
      
    # Prediction using gini 
    y_pred_gini = prediction(X_test, clf_gini) 
    cal_accuracy(y_test, y_pred_gini) 
      
    print("Results Using Entropy:") 
    # Prediction using entropy 
    y_pred_entropy = prediction(X_test, clf_entropy) 
    cal_accuracy(y_test, y_pred_entropy) 

In [51]:
# Run the entire code
while True:
  try:
    var = float(input("Enter the training data split number: "))
    
    if var <= 0.99 and var >= 0.01:
      main(var) 
    
  except ValueError:
    print("This process has been stopped.")
    print("To continue rerun and enter a number between 0.01 and 0.99")
    break

Enter the training data split number: 0.72


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Results Using Gini Index:
Predicted values:
[1 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1
 1 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 1 0 1 1 0 0
 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0 0 0 1 0 0
 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0]
Confusion Matrix:  [[131  16]
 [ 33  70]]
Accuracy : 80.40%
Report :                precision    recall  f1-score   support

           0       0.80      0.89      0.84       147
           1       0.81      0.68      0.74       103

    accuracy                           0.80       250
   macro avg       0.81      0.79      0.79       250
weighted avg       0.81      0.80      0.80       250

Results Using Entropy:
Predicted values:
[1 1 0 1 0

###Evaluation
Improvements for future renditions: 
- When producing the ages for null values in the dataset, the ages could have been grouped by class and gender and class of the individual, rather than just using gender. This may produce a more accurate number. 
- Use Gini Idex instead of Entropy?

###Results
This is the results from the decision tree model, displaying the accuracy outcomes for the changing variables

Train Data Split|Criterion|Max Depth|Misclassified|Accuracy
---|---|---|---|---
0.6|Entropy|3|70|0.8039
0.6|Entropy|5|68|0.8095
0.6|Gini|3|67|0.8123
0.6|Gini|5|77|0.7843
0.7|Entropy|3|48|0.8209
0.7|Entropy|5|48|0.8209
0.7|Gini|3|48|0.8209
0.7|Gini|5|49|0.8172

###Conclusion

###Ethical Justification

- potential misuses of the technique
- utilitarian view?

###Video Pitch
*Attached to the final document*